In [1]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd
from os import listdir
import cv2
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from PIL import Image  
import PIL  
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image  as mpimg
import inception_cnn
from inception_cnn import inception_net
import data_segment
from data_segment import split_data,data_visualize
import baseline_cnn
from baseline_cnn import model_net
#!pip install opencv-python
#!pip install Pillow

C:\Users\Mridu\Anaconda3\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Mridu\Anaconda3\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Mridu\Anaconda3\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Mridu\Anaconda3\envs\tf\lib\site-packages\tensorflow\python\framework\dty

In [12]:
def main(source):
    
    #import zipfile
    #file_name="C:/Users/Aface_mask\observations-master.zip"
    #with zipfile.ZipFile(file_name, 'r') as zip: 
    # printing all the contents of the zip file 
    #zip.printdir() 
  
    # extracting all the files 
    print('Extracting all the files now...') 
    zip.extractall("observations-master/experiements/data/") 
    print('Done!')
    
    source_mask = source + "/with_mask"
    source_nomask= source+"/without_mask"
    path="clsfr_data_final1"
    yes_dir="Yes"
    No_dir="No"
    split_size = 0.8
    
    
    yes_dir=split_data(source_mask,path, yes_dir, split_size)
    no_dir=split_data(source_nomask, path, No_dir, split_size)
    
    
    yes_len=len(os.listdir(yes_dir[0]))+len(os.listdir(yes_dir[1]))
    no_len=len(os.listdir(no_dir[0]))+len(os.listdir(no_dir[1]))
    
    source_mask_len=len(os.listdir(source_mask))
    source_nomask_len=len(os.listdir(source_nomask))
    
    
    assert((yes_len== source_mask_len)and (no_len==source_nomask_len))
    print("Succesfully Generated dataset for model!")
    
    data_visualize(yes_dir,no_dir)
    
    model= model_net()
    
    TRAINING_DIR = "C:/Users/Mridu/Aface_mask\clsfr_data_final1\Train"
    train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

    train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
    VALIDATION_DIR = "C:/Users/Mridu/Aface_mask\clsfr_data_final1\Test"
    validation_datagen = ImageDataGenerator(rescale=1.0/255)

    validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))
    checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
    
    #with tf.device('/gpu:0'):
    
    model.fit_generator(train_generator,epochs=20,validation_data=validation_generator,verbose=1)
        
    model.save_weights('pretrained_weights.h5')
    
        #filename='finalized_model1.sav'
        #pickle.dump(model,open(filename,'wb'))
    

In [13]:
main("observations-master/experiements/data")

labels_dict={0:'without_mask',1:'waah Modiji Waah'}
color_dict={0:(0,0,255),1:(0,255,0)}

size = 4
webcam = cv2.VideoCapture(0) #Use camera 0

# We load the xml file
classifier = cv2.CascadeClassifier('C:/Users/Mridu/Aface_mask\haarcascade_frontalface_default.xml')

while True:
    (rval, im) = webcam.read()
    im=cv2.flip(im,1,1) #Flip to act as a mirror

    # Resize the image to speed up detection
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))

    # detect MultiScale / faces 
    faces = classifier.detectMultiScale(mini)

    # Draw rectangles around each face
    for f in faces:
        (x, y, w, h) = [v * size for v in f] #Scale the shapesize backup
            #Save just the rectangle faces in SubRecFaces
        face_img = im[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(150,150))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        #print(result)

        label=np.argmax(result,axis=1)[0]

        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

    # Show the image
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    # if Esc key is press then break out of the loop 
    if key == 27: #The Esc key
        break
    # Stop video
webcam.release()

    # Close all started windows
cv2.destroyAllWindows()

Succesfully Generated dataset for model!
The number of images with facemask in the training set labelled 'yes': 552
The number of images with facemask in the test set labelled 'yes': 138
The number of images without facemask in the training set labelled 'no': 548
The number of images without facemask in the test set labelled 'no': 138
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Found 1100 images belonging to 2 classes.
Found 276 images belonging to 2 classes.
Epoch 1/20
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
110/110 [==============================] - 147s 1s/step - loss: 0.7350 - acc: 0.5173 - val_loss: 0.6835 - val_acc: 0.5435
Epoch 2/20
110/110 [==============================] - 141s 1s/step - loss: 0.5303 - acc: 0.7536 - val_loss: 0.2575 - val_acc: 0.9674
Epoch 3/20
110/110 [==============================] - 148s 1s/step - loss: 0.3654 - acc: 0.8700 - val_